# Clustering (apprentissage non-supervisé)

Pour de nombreux problèmes, il existe des bases de données volumineuses, mais non annotées. Dans ce cas, on souhaite souvent trouver une partition des données qui donne des groupes homogènes. Il existe de nombreux algorithmes de partitionnement de données (*clustering* en anglais) qui utilisent des critères différents d'*homogénéité* et qui, généralement, laisse le choix du nombre de groupes (appelés *clusters*) à l'utilisateur. 

L'objectif de ce TP est de se familiariser avec les principaux algorithmes de clustering, et des critères qui permettent de sélectionner le nombre de clusters (puisqu'en l'absence d'annotations, on ne peut sélectionner les hyper-paramètres par validation croisée).

Crédit : notebook inspiré d'un TP de Nicolas Enjalbert Courrech (INRAE, MIAT Toulouse).

### Notations

Soit la matrice de données $X \in \mathbb{R}^{n \times p}$ exprimant $p$ variables pour $n$ individus. Le but est de trouver une partition $P = \{P_1, \ldots, P_K\}$ des $n$ individus où $P_k \subset P$ est l'ensemble des individus du groupe $k$. On note $\mu_k \in \mathbb{R}^p$ le barycentre des individus contenus dans le groupe $k$ tels que $\forall k \in \{1, ..., K\}, \mu_k = \frac{1}{n_k} \sum_{i \in P_k} X_{i,\cdot}$ avec $n_k = |P_k|$ et $X_{i,\cdot}$ la $i$-eme ligne de la matrice $X$.

### Librairies utilisées

Dans ce TP, on utilisera les librairies `pandas` et `numpy` pour la lecture et le pré-traitement des données, `sklearn` et `scipy` pour l'implémentation et l'interprétation des algorithmes de clustering, et `matplotlib` pour la visualisation des données et des résultats.

In [ ]:
import pandas as pd 
import numpy as np 

import sklearn
from sklearn import decomposition
from sklearn import cluster
from sklearn import metrics

from scipy.cluster.hierarchy import dendrogram

import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import matplotlib.cm as cm

### Préparation du jeu de données 

Nous allons utiliser les données Iris de Fisher. Notez que les annotations / variables à expliquer (*i.e.*, les espèces d'Iris) vont être utiles pour la visualisation des données et l'interprétation des résultats, mais qu'elles ne seront jamais utilisées par les algorithmes de clustering. 

In [ ]:
# Credits: https://www.angela1c.com/projects/iris_project/downloading-iris/
csv_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
# using the attribute information as the column names
col_names = ['Sepal_Length','Sepal_Width','Petal_Length','Petal_Width','Species']
iris =  pd.read_csv(csv_url, names = col_names)
iris

In [ ]:
y_true = iris[["Species"]]
y_true

On simule un jeu de données non annotées.

In [ ]:
X = iris.drop(["Species"], axis = 1)
X

On utilise une analyse en composantes principales (ACP) pour réduire la dimension des données, ce qui facilite leur visualisation. On verra comment fonctionne l'ACP dans un autre TP.

In [ ]:
pca = decomposition.PCA(n_components = 2)
pca_X = pca.fit_transform(X)

In [ ]:
classes = np.unique(y_true)
str2int = dict((k, v) for (v, k) in enumerate(classes))
colors =  np.vectorize(
# <CORRECTION>
    str2int.get
    )(
    y_true
# </CORRECTION>
)

fig, ax = plt.subplots()
scatter = plt.scatter(pca_X[:, 0], pca_X[:, 1], c=colors, cmap='plasma')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend(handles=scatter.legend_elements()[0], 
           labels=list(classes))
plt.show()

## 1. Algorithmes de clustering

In [ ]:
clusters = {
    'kmeans': {},
    'hier': {},
    'dbscan': {}
}

### 1.1 K-means

Utilisez la classe `sklearn.cluster.KMeans` pour catégoriser les données pour $K \in \{2, \ldots, 11\}$ clusters. Visualisez les clusters obtenus en utilisant la projection sur les deux premières composantes principales. Intuitivement, quel clustering vous semble-t-il le meilleur ? Sur quels critères vous basez vous ?

In [ ]:
# <CORRECTION>
k_min, k_max = 2, 11
n_clustering = k_max - k_min + 1

for k in range(k_min, k_max + 1): 
    kmeans = cluster.KMeans(n_clusters=k).fit(X)
    clusters['kmeans']['k=' + str(k)] = kmeans
# </CORRECTION>

In [ ]:
n_rows = 2
n_cols = n_clustering // n_rows
fig, ax = plt.subplots(n_rows, n_cols, figsize=(15, 10))
# <CORRECTION>
for i, kmeans_cluster in enumerate(clusters['kmeans'].values()):
    ax[i // n_cols, i % n_cols].scatter(pca_X[:, 0], pca_X[:, 1], c=kmeans_cluster.labels_,  cmap='plasma')
# </CORRECTION>
plt.show()

### 1.2 Clustering Hierarchique Ascendant

Le clustering hiérarchique ascendant est un algorithme de clustering dont le nombre de clusters diminue à chaque itération. A l'initialisation, le nombre de clusters $K$ est égal au nombre de données $n$. Ainsi, toutes les données appartiennent à des clusters différents. A chaque itération, les clusters sont fusionnés selon un critère de similarité. A l'itération $t$, il y a donc $n - t$ clusters. L'algorithme s'arrête lorsque toutes les données appartiennent à un seul cluster.

Utilisez la classe `sklearn.cluster.AgglomerativeClustering` pour catégoriser les données de manière hiérarchique avec `n_clusters=None` et `distance_threshold=0`. 

Les clusters sont centenus dans l'attribut `labels_` : sont-ils informatifs ? Recommencez en augmentant la `distance_threshold`. Comment choisir cette distance ? Le choix de la distance est-il équivalent au choix du nombre de clusters ?

Utilisez à nouveau `distance_threshold=0` : que contiennent les attributs `children_` et `distances_` ? Comment évoluent les distances au cours des itérations ?

In [ ]:
# <CORRECTION>
hier_clustering = cluster.AgglomerativeClustering(n_clusters=None, distance_threshold=0)
hier_clustering.fit(X)
# </CORRECTION>

In [ ]:
# <CORRECTION>
print(len(np.unique(hier_clustering.labels_)))
# </CORRECTION>

In [ ]:
# <CORRECTION>
print(f"shape: {hier_clustering.children_.shape}, min value: {hier_clustering.children_.min()}, max value: {hier_clustering.children_.max()}")
hier_clustering.children_[:10, :]
# </CORRECTION>

In [ ]:
fig = plt.figure()
# <CORRECTION>
plt.plot(hier_clustering.distances_)
plt.xlabel('Itérations')
# </CORRECTION>
plt.ylabel('Distance entre les clusters fusionnés')
plt.show()

Un très bon outil de visualisation et d'aide à la décision est le dendogramme. Un dendogramme est un diagramme qui illustre l'arrangement des clusters. Il permet de visusaliser l'ascendance des clusters, et la distance entre les clusters fusionnés.

In [ ]:
# Crédit : https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_dendrogram.html

def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count
    
    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

In [ ]:
# <CORRECTION>
dist_k3 = 8

plot_dendrogram(hier_clustering, truncate_mode="level", p=3)
plt.hlines(dist_k3, xmin=0, xmax=160, color='black', linestyle='--')
plt.xlabel("Number of points in node (or index of point if no parenthesis).")
plt.ylabel("Distance between merged clusters")
# <C/ORRECTION>
plt.title("Hierarchical Clustering Dendrogram")
plt.show()

Tracez sur le dendogramme la `distance_threshold` qui permet d'obtenir 3 clusters. Utilisez le clustering hiérarchique avec cette distance. Calculez le nombre de données par clusters à partir du dendogramme. Vérifiez en Python. 

In [ ]:
# <CORRECTION>
hier_clustering_k3 = cluster.AgglomerativeClustering(n_clusters=None, distance_threshold=dist_k3)
hier_clustering_k3.fit(X)
print('n_clusters: ', len(np.unique(hier_clustering_k3.labels_)))
# </CORRECTION>

In [ ]:
# <CORRECTION>
fig = plt.figure()
plt.scatter(pca_X[:, 0], pca_X[:, 1], c=hier_clustering_k3.labels_, cmap='plasma')
plt.show()
# </CORRECTION>

Testez la méthode de clustering hiérarchique pour 3 clusters et d'autres critères de similarité.

In [ ]:
k = 3

for linkage in ['ward', 'average', 'complete', 'single']:
    cl = cluster.AgglomerativeClustering(n_clusters=k, linkage=linkage).fit(X)
    clusters['hier'][linkage] = cl

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(8,8))

# <CORRECTION>
ax[0, 0].scatter(pca_X[:, 0], pca_X[:, 1], c=clusters['hier']['ward'].labels_, cmap='plasma')
ax[0, 1].scatter(pca_X[:, 0], pca_X[:, 1], c=clusters['hier']['average'].labels_, cmap='plasma')
ax[1, 0].scatter(pca_X[:, 0], pca_X[:, 1], c=clusters['hier']['complete'].labels_, cmap='plasma')
ax[1, 1].scatter(pca_X[:, 0], pca_X[:, 1], c=clusters['hier']['single'].labels_, cmap='plasma')

ax[0,0].set_title("Ward")
ax[0,1].set_title("Average")
ax[1,0].set_title("Complete")
ax[1,1].set_title("Single")
# </CORRECTION>

plt.show()

### 1.3 DBSCAN (Density-Based Spatial Clustering of Applications with Noise)

DBSCAN est un algorithme de clustering basé sur la spacialité qui cherche à trouver des zones à forte densité séparées par des zones à faible densité. La recherche se fait de proche en proche en parcourant une boule de rayon `eps` centrée sur les données. Si aucun voisin n'est inclus dans la boule, alors la zone courante est considérée comme une zone à faible densité, et aucun cluster n'est attribué à la donnée (cluster -1 dans `sklearn.cluster.DBSCAN`).

Utilisez `sklearn.cluster.DBSCAN` pour différentes tailles de voisinage. Que se passe-t-il quand la taille est trop petite ? Trop grande ?

In [ ]:
# <CORRECTION>
n_clustering = 10
eps = np.linspace(0.1, 2, n_clustering)

for i, e in enumerate(eps): 
    cl_dbscan = cluster.DBSCAN(eps=e).fit(X)
    clusters['dbscan']['eps=' + str(e)] = cl_dbscan
# </CORRECTION>

In [ ]:
n_rows = 2
n_cols = n_clustering // n_rows

norm = Normalize(vmin=-1, vmax=3)

fig, ax = plt.subplots(n_rows, n_cols, figsize=(15, 10))
for i, dbscan_clusters in enumerate(clusters['dbscan'].values()):
    ax[i // n_cols, i % n_cols].scatter(pca_X[:, 0], pca_X[:, 1], c=dbscan_clusters.labels_,  cmap='plasma', norm=norm)
plt.show()

Représentez le nombre de données par cluster avec un histogramme.

In [ ]:
# <CORRECTION>
dbscan_labels = dict((k, cluster.labels_) for k, cluster in clusters['dbscan'].items())
pd.DataFrame(dbscan_labels).hist(figsize=(10,10))
# </CORRECTION>

## 2. Sélection de modèle 

Comment sélectionner les hyper-paramètres des algorithmes de clustering, qui conditionnent le nombre de clusters ? Prenons l'exemple des K-moyennes. Cela a-t-il du sens de sélectionner le nombre de clusters qui minimise la fonction objective (l'inertie intra-classe) ? Faites un graphe de l'inertie intra-classe (voir l'attribut `inertia_` de `sklearn.cluster.KMeans`) en fonction du nombre de clusters.

In [ ]:
# <CORRECTION>

kmeans_inertia = dict((k, cluster.inertia_) for k, cluster in clusters['kmeans'].items())

fig, ax = plt.subplots()
plt.plot(list(kmeans_inertia.values()))
ax.set_xticks(list(range(len(kmeans_inertia.keys()))))
ax.set_xticklabels(list(kmeans_inertia.keys()))
plt.xlabel('Number of clusters')
plt.ylabel('Intra-class inertia')
plt.show()

# </CORRECTION>

### 2.1 Silhouette

Le coefficient de la Silhouette mesure l'homogénéité des clusters : sont-ils bien regroupés et éloignés des autres ? Pour les K-moyennes, puis pour chaque méthode de clustering, comparer le coefficient de la silhouette afin de sélectionner le modèle avec le meilleur résultat.

In [ ]:
# <CORRECTION>
silhouette_kmeans = {}
for params, cluster in clusters['kmeans'].items():
    silhouette_kmeans[params] = metrics.silhouette_score(X, cluster.labels_, metric="euclidean")
# <C/ORRECTION>

In [ ]:
fig, ax = plt.subplots()
plt.plot(list(silhouette_kmeans.values()))
k_list = list(silhouette_kmeans.keys())
ax.set_xticks(list(range(len(silhouette_kmeans.keys()))))
ax.set_xticklabels(k_list)
plt.title("Valeur du score silhouette en fonction du nombre de cluster")
plt.show()

Le score silouhette pour un clustering est la moyenne du score silouhette pour chaque donnée du clustering. Il peut être utile de visualiser le score pour chaque donnée également.

In [ ]:
n_clusters = 2
cluster_labels = clusters['kmeans'][f'k={n_clusters}'].labels_
sample_silhouette_values = metrics.silhouette_samples(X, cluster_labels)

y_lower = 10
fig, ax1 = plt.subplots(figsize=(8, 6))
for i in range(n_clusters):
    # Aggregate the silhouette scores for samples belonging to
    # cluster i, and sort them
    ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]

    ith_cluster_silhouette_values.sort()

    size_cluster_i = ith_cluster_silhouette_values.shape[0]
    y_upper = y_lower + size_cluster_i

    color = cm.nipy_spectral(float(i) / n_clusters)
    ax1.fill_betweenx(
        np.arange(y_lower, y_upper),
        0,
        ith_cluster_silhouette_values,
        facecolor=color,
        edgecolor=color,
        alpha=0.7,
    )

    # Label the silhouette plots with their cluster numbers at the middle
    ax1.text(-0.025, y_lower + 0.5 * size_cluster_i, str(i))


    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    y_lower = y_upper + 10

plt.show()

In [ ]:
silhouette = {}
for cluster_name in clusters.keys():
    silhouette[cluster_name] = {}
    for cluster_param, cluster in clusters[cluster_name].items():
        if len(np.unique(cluster.labels_)) > 1:
            # <CORRECTION>
                silhouette[cluster_name][cluster_param] = metrics.silhouette_score(X, cluster.labels_, metric="euclidean")
            # </CORRECTION>
        else:
            # <CORRECTION>
            silhouette[cluster_name][cluster_param] = np.nan
            # </CORRECTION>

In [ ]:
def plot_hist(score_dict):
    labels = list(score_dict['kmeans'].keys()) + list(score_dict['hier'].keys()) + list(score_dict['dbscan'].keys())
    x = np.arange(len(labels))
    values = []
    colors = []
    for cluster_name in score_dict:
        score_values = [score_dict[cluster_name][k] for k in score_dict[cluster_name]]
        values.extend(score_values)
        
        # Assign colors by cluster type
        if cluster_name == 'kmeans':
            colors.extend(['blue'] * len(score_values))
        elif cluster_name == 'hier':
            colors.extend(['orange'] * len(score_values))
        elif cluster_name == 'dbscan':
            colors.extend(['green'] * len(score_values))
    
    fig, ax = plt.subplots(figsize=(15, 6))
    bars = ax.bar(x, values, color=colors)
    
    # Customize x-axis
    ax.set_xticks(x)
    ax.set_xticklabels(labels, rotation=45, ha='right')
    
    # Add legend
    ax.legend(handles=[
        plt.Rectangle((0,0),1,1, color='blue', label='K-means'),
        plt.Rectangle((0,0),1,1, color='orange', label='Hierarchical'),
        plt.Rectangle((0,0),1,1, color='green', label='DBSCAN')
    ], loc='upper right', fontsize=15)
    
    # Add labels and title
    ax.set_ylabel('Score')
    plt.tight_layout()
    return ax

In [ ]:
ax = plot_hist(silhouette)
ax.set_title('Silhouette Score by Method and Parameters')
plt.show()

### 2.2 Davies Bouldin Score

Le score de Davies Bouldin mesure pour chaque cluster sa similarité avec le cluster le plus proche, mesuré comme le ratio de l'inertie intra-classe sur l'inertie inter-classe.

In [ ]:
dbscore = {}
for cluster_name in clusters.keys():
    dbscore[cluster_name] = {}
    for cluster_param, cluster in clusters[cluster_name].items():
        if len(np.unique(cluster.labels_)) > 1:
            # <CORRECTION>
            dbscore[cluster_name][cluster_param] = metrics.davies_bouldin_score(X, cluster.labels_)
            # </CORRECTION>
        else:
            # <CORRECTION>
            dbscore[cluster_name][cluster_param] = np.nan
            # </CORRECTION>

In [ ]:
ax = plot_hist(dbscore)
ax.set_title('Davies Bouldin Score by Method and Parameters')
plt.show()

### 2.3 Table de contingence

La table de contingence est un outil statistique simple qui permet de voir rapidement si les individus sont classé dans la même classe entre deux partitions. Deux partitions similaires donneront une matrice diagonale. La table de contingence peut donc être utile pour comparer deux algorithmes de clustering. Néanmoins, sans annotation, elle ne permet pas de sélectionner les hyper-paramètres. 

In [ ]:
print("Nb clusters kmeans: ", len(np.unique(clusters['kmeans']['k=2'].labels_)))
print("Nb clusters hierarchical clustering: ", len(np.unique(clusters['hier']['ward'].labels_)))

# <CORRECTION>
contingency = metrics.cluster.contingency_matrix(clusters['kmeans']['k=2'].labels_, clusters['hier']['ward'].labels_)
# </CORRECTION>
contingency

### 2.4 Indice de Rand

L'indice de Rand est un score qui permet de mesurer la similitude entre deux partitions. Notons $P_1$ et $P_2$ deux partitions des données désignées par leur indice $i \in \{1, \ldots, n\}$. Pour calculer l'indice de Rand, on calcule le nombre $a$, $b$, $c$, et $d$ de paires d'éléments $(i,j)$ qui sont : 
* dans le même groupe dans $P_1$ et le même groupe dans $P_2$,
* dans le même groupe dans $P_1$ et dans un groupe différent dans $P_2$,
* dans un groupe différent dans $P_1$ et le même groupe dans $P_2$,
* dans un groupe différent dans $P_1$ et dans un groupe différent dans $P_2$, respectivement.

L'indice de Rand est alors $$ RI = \frac{a+d}{a+b+c+d}.$$

Par rapport à la table de contigence, quel est l'avantage de cette métrique ?

In [ ]:
rand_score = {}
for cluster_name in clusters.keys():
    rand_score[cluster_name] = {}
    for cluster_param, cluster in clusters[cluster_name].items():
        # <CORRECTION>
        rand_score[cluster_name][cluster_param] = metrics.rand_score(cluster.labels_, y_true.to_numpy().reshape(-1))
        # </CORRECTION>

In [ ]:
ax = plot_hist(rand_score)
ax.set_title('Rand Score by Method and Parameters')
plt.show()

Les critères de sélection s'accordent-ils ?

In [2]:
### <CORRECTION> ###
import re
# transformation de cet énoncé en version étudiante

fname = "6-notebook-clustering-corr.ipynb" # ce fichier
fout  = fname.replace("-corr","")

# print("Fichier de sortie: ", fout )

f = open(fname, "r")
txt = f.read()

f.close()


f2 = open(fout, "w")
f2.write(re.sub("<CORRECTION>.*?(</CORRECTION>)"," TODO ",\
    txt, flags=re.DOTALL))
f2.close()

### </CORRECTION> ###